In [ ]:
import pandas as pd
import seaborn as sns

path_main_dir = ".."

df = pd.read_csv(path_main_dir + "/output/results.csv")
df.head()

In [ ]:
default_params = {
    #embedding computation
    'cleora_n_iter': 5,
    'cleora_dim': 1024,

    #dataset preparation
    'train_test_split': 0.2,
}

In [ ]:
import os
import matplotlib.pyplot as plt

os.makedirs(f"{path_main_dir}/output/plots/experiments", exist_ok=True)

for dataset_name in ['fb-pages', 'walmart-trips', 'trivago-clicks-continents']:
    default_params['dataset_name'] = dataset_name
    for param in ['cleora_n_iter', 'cleora_dim']:
        data = df
        params = default_params.copy()
        params.pop(param)

        for k, v in params.items():
            data = data.loc[data[k] == v]

        data.sort_values(by=[param], inplace=True)

        fig, axes = plt.subplots(3, 2, sharex=True, sharey='row')
        plt.rcParams.update({'font.size': 15})

        fig.set_size_inches((12, 6))
        fig.suptitle(params["dataset_name"])
        fig.supxlabel(param)
        fig.supylabel('value')

        ax1, ax2 = axes[0]
        ax1.set_title('clique')
        ax2.set_title('star')

        measures = ['f1_micro', 'f1_macro', 'log_loss']
        colors = ['r', 'g', 'b']

        for measure, (ax1, ax2), color in zip(measures, axes, colors):
            data_final = data.loc[data['algo'] == 'clique']
            ax1.errorbar(param, measure, yerr=f"std_{measure}", label=measure, color=color, data=data_final)

            data_final = data.loc[data['algo'] == 'star']
            ax2.errorbar(param, measure, yerr=f"std_{measure}", label=measure, color=color, data=data_final)

            ax1.legend()
            ax2.legend()

        plt.tight_layout()
        plt.savefig(f"{path_main_dir}/output/plots/experiments/{dataset_name}_{param}")
        plt.show()